In [143]:
#𝐕𝐢𝐫𝐭𝐮𝐚𝐥 𝐄𝐱𝐡𝐢𝐛𝐢𝐭𝐢𝐨𝐧 𝐆𝐞𝐧𝐞𝐫𝐚𝐭𝐨𝐫

# This project aims to create a virtual exhibition generator that uses the Spotify API 
# and the Art Institute of Chicago's API. The goal is to allow users to select art genres
# and receive an accompanying music playlist to enhance their viewing experience.

In [144]:
#Importing necessary tools
# Spotipy: For accessing the Spotify API.
# Pandas: For data handling and manipulation.
# Urllib: For making HTTP requests to APIs.
# IPython.display: For displaying images and media in the Notebook.

import spotipy
import spotipy.util as util
import json
import webbrowser
import pandas as pd
import urllib.request
from IPython.display import display, Image

In [145]:
#Accessing Spotify API 

cred = "spotify_keys.json"
with open(cred, "r") as key_file:
    api_tokens = json.load(key_file)

client_id = api_tokens['client_id']
client_secret = api_tokens['client_secret']
redirectURI = api_tokens['redirect']
username = api_tokens['username']


In [146]:
client_id = api_tokens['client_id']
client_secret = api_tokens['client_secret']
redirectURI = api_tokens['redirect']
username = api_tokens['username'] 

In [147]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirectURI)

sp = spotipy.Spotify(auth=token)

In [148]:
# Artwork data is loaded from a CSV file of data from the AIC Github,containing information 
# about their most popular art pieces. This is the first step in my data collection process.

art_data = pd.read_csv("AIC_data.csv")

In [149]:
display(art_data)

,id,title,main_reference_number,department_title,artist_title
0,869,The Watermill with the Great Red Roof,1894.1031,Painting and Sculpture of Europe,Meindert Hobbema
1,2189,Ready-to-Wear,1956.137,Arts of the Americas,Stuart Davis
2,2816,Interior at Nice,1956.339,Modern Art,Henri Matisse
3,4081,Gian Lodovico Madruzzo,1929.912,Painting and Sculpture of Europe,Giovanni Battista Moroni
4,4428,Mère Grégoire,1930.78,Painting and Sculpture of Europe,Gustave Courbet
...,...,...,...,...,...
417,234004,Untitled 72–12–A,2016.62,Contemporary Art,"Chung, Sang-Hwa"
418,234433,Involvement Series,2016.66,Contemporary Art,Wanda Pimentel
419,234781,Christ Carrying the Cross,2016.15,Painting and Sculpture of Europe,Sebastiano del Piombo
420,234972,Small Town by Day (Badische Kleinstadt bei Tage),2016.46,Modern Art,Georg Scholz


In [150]:
# Unique department titles are extracted from the art data, providing insight into how the artworks are categorised.

departments = art_data["department_title"].unique()

print("Department Titles",departments)

Department Titles ['Painting and Sculpture of Europe' 'Arts of the Americas' 'Modern Art'
 'Contemporary Art' 'Arts of Asia' 'Photography and Media'
 'Prints and Drawings' 'Applied Arts of Europe' 'Textiles'
 'Arts of Africa' 'Architecture and Design' nan
 'Arts of the Ancient Mediterranean and Byzantium']


In [151]:
# I then make a request to the Art Institute API to retrieve additional artwork details, including image IDs and classifications, 
# which enhances the dataset I am working with, beyond the CSV file, which is missing the image ids i need.

url = "https://api.artic.edu/api/v1/artworks?fields=id,title,artist_display,date_display,classification_titles,department_title,image_id"
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)
all_art_data= json.loads(response.read())
all_art_data

{'pagination': {'total': 126319,
  'limit': 12,
  'offset': 0,
  'total_pages': 10527,
  'current_page': 1,
  'next_url': 'https://api.artic.edu/api/v1/artworks?page=2&fields=id%2Ctitle%2Cartist_display%2Cdate_display%2Cclassification_titles%2Cdepartment_title%2Cimage_id'},
 'data': [{'id': 25770,
   'title': 'The Crucifixion',
   'date_display': 'c. 1370',
   'artist_display': 'Francescuccio Ghissi (Italian, active 1359–1395)',
   'department_title': 'Painting and Sculpture of Europe',
   'classification_titles': ['tempera',
    'paint',
    'Italian',
    'painting',
    'european painting'],
   'image_id': '4c862fdf-89d6-7c37-ccd2-3c5ee85e5161'},
  {'id': 25753,
   'title': 'Two Putti',
   'date_display': '1490/1510',
   'artist_display': 'Matteo di Giovanni (Italian, c. 1430–1495)',
   'department_title': 'Painting and Sculpture of Europe',
   'classification_titles': ['tempera',
    'Italian',
    'paint',
    'oil paintings (visual works)',
    'painting',
    'european painting'

In [179]:
# Analysing artword data

for artwork in all_art_data['data'][:20]: 
    
    print("Title:", artwork['title'])
    print("Artist:", artwork.get('artist_display',))
    print("Date:", artwork.get('date_display'))
    print("Classification:", artwork.get('classification_titles'))
    print("Department:", artwork.get('department_title'))
    print("\n")

Title: The Crucifixion
Artist: Francescuccio Ghissi (Italian, active 1359–1395)
Date: c. 1370
Classification: ['tempera', 'paint', 'Italian', 'painting', 'european painting']
Department: Painting and Sculpture of Europe


Title: Two Putti
Artist: Matteo di Giovanni (Italian, c. 1430–1495)
Date: 1490/1510
Classification: ['tempera', 'Italian', 'paint', 'oil paintings (visual works)', 'painting', 'european painting']
Department: Painting and Sculpture of Europe


Title: Fragment from Christ Carrying the Cross: Saint John the Evangelist
Artist: Jean Hey (Master of Moulins; Netherlandish, active in France, c. 1480-c.1504)
Date: c. 1500
Classification: ['painting', 'oil on panel', 'oil paintings (visual works)', 'paint', 'fragment', 'french', 'european painting']
Department: Painting and Sculpture of Europe


Title: Elements I
Artist: Nancy Hemenway Barton (American, 1920–2008)
Mouse Island, Maine, and Washington, DC, United States
Date: 1978
Classification: ['textile', 'needlework (visual 

In [199]:
# Convert the JSON data to a DataFrame
artworks_df = pd.json_normalize(all_art_data['data'])

# Filter to include only records with an image_id
artworks_with_images_df = artworks_df[artworks_df["image_id"].notna()]

# Display the first few rows of the filtered DataFrame
display(artworks_with_images_df.head())
print(f"Total artworks with images: {len(artworks_with_images_df)}")


,id,title,date_display,artist_display,department_title,classification_titles,image_id
0,99766,Untitled (Butterfly Habitat),c. 1940,"Joseph Cornell\nAmerican, 1903–1972",Modern Art,"[sculpture, Surrealism Highlights, constructio...",1221a4a5-0b72-41df-3624-98abb53cf3a9
1,181145,Horse and Rider,1630,"Hans Ludwig Kienle\nGerman, 1591–1653\nUlm, Ge...",Applied Arts of Europe,"[statuette, european decorative arts, silver, ...",1dcb1fd9-8254-8098-586c-e5b117c00871
2,156442,Coffer,1700/20,"Attributed to André-Charles Boulle (French, 16...",Applied Arts of Europe,"[chest (case furniture), case furniture, furni...",be169735-ea1c-7440-c438-bafd24021119
3,217295,Statue of Young Dionysos,100 BCE-100 CE,Hellenistic or Roman; Eastern Mediterranean,Arts of the Ancient Mediterranean and Byzantium,"[sculpture, vessel]",3f3ba9a2-24e0-435f-64e7-31114121b3be
4,198905,Still Life with Ostrich Egg Cup and the Whitfi...,c. 1670,"Pieter Gerritsz. van Roestraeten (Dutch, 1630–...",Painting and Sculpture of Europe,"[painting, still life, paint, oil paintings (v...",a3bfd218-9b88-f59b-5ade-f39b45f3ec4f


Total artworks with images: 12


In [200]:
# Check unique department titles in the filtered data
unique_departments = artworks_with_images_df["department_title"].unique()
print("Departments with images available:", unique_departments)

Departments with images available: ['Modern Art' 'Applied Arts of Europe'
 'Arts of the Ancient Mediterranean and Byzantium'
 'Painting and Sculpture of Europe']


In [201]:
# Filter for artworks in "Modern Art" and "Contemporary Art" departments
departments_of_interest = ["Modern Art", "Contemporary Art", "Textiles", "Photography and Media", "Prints and Drawings"]
filtered_departments_df = artworks_with_images_df[
    artworks_with_images_df["department_title"].isin(departments_of_interest)
]

# Display a sample to verify and check the count
display(filtered_departments_df.head())
print(f"Total artworks from selected departments with images: {len(filtered_departments_df)}")

,id,title,date_display,artist_display,department_title,classification_titles,image_id
0,99766,Untitled (Butterfly Habitat),c. 1940,"Joseph Cornell\nAmerican, 1903–1972",Modern Art,"[sculpture, Surrealism Highlights, constructio...",1221a4a5-0b72-41df-3624-98abb53cf3a9


Total artworks from selected departments with images: 1


In [202]:
# Function to display a sample of artworks with images from all departments
def display_all_departments_artworks(filtered_art_data, num_samples=5):
    for _, artwork in filtered_art_data.head(num_samples).iterrows():
        print(f"Title: {artwork['title']}")
        print(f"Artist: {artwork['artist_display']}")
        print(f"Department: {artwork['department_title']}")
        display(Image(url=f"https://www.artic.edu/iiif/2/{artwork['image_id']}/full/843,/0/default.jpg", width=300))
        print("\n")

# Display a sample of artworks from all departments with images
display_all_departments_artworks(artworks_with_images_df)

Title: Untitled (Butterfly Habitat)
Artist: Joseph Cornell
American, 1903–1972
Department: Modern Art




Title: Horse and Rider
Artist: Hans Ludwig Kienle
German, 1591–1653
Ulm, Germany
Department: Applied Arts of Europe




Title: Coffer
Artist: Attributed to André-Charles Boulle (French, 1642–1732)
France, Paris
Department: Applied Arts of Europe




Title: Statue of Young Dionysos
Artist: Hellenistic or Roman; Eastern Mediterranean
Department: Arts of the Ancient Mediterranean and Byzantium




Title: Still Life with Ostrich Egg Cup and the Whitfield Heirlooms
Artist: Pieter Gerritsz. van Roestraeten (Dutch, 1630–1700)
Department: Painting and Sculpture of Europe


In [203]:
# Count the number of artworks per department
department_counts = artworks_with_images_df["department_title"].value_counts()

# Display the counts for each department
print(department_counts)

department_title
Applied Arts of Europe                             9
Modern Art                                         1
Arts of the Ancient Mediterranean and Byzantium    1
Painting and Sculpture of Europe                   1
Name: count, dtype: int64


In [204]:
#Accessing Spotify API 

cred = "spotify_keys.json"
with open(cred,"r") as key_file:
    api_tokens = json.load(key_file)

key_file.close()

client_id = api_tokens['client_id']
client_secret = api_tokens['client_secret']
redirectURI = api_tokens['redirect']
username = api_tokens['username'] 

scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirectURI)

In [205]:
sp = spotipy.Spotify(auth=token)

In [206]:
# Available genre seeds from Spotify are fetched, which will later be used to 
# generate music recommendations based on the selected art categories.

genre_seeds = sp.recommendation_genre_seeds()

In [207]:
genre_seeds

{'genres': ['acoustic',
  'afrobeat',
  'alt-rock',
  'alternative',
  'ambient',
  'anime',
  'black-metal',
  'bluegrass',
  'blues',
  'bossanova',
  'brazil',
  'breakbeat',
  'british',
  'cantopop',
  'chicago-house',
  'children',
  'chill',
  'classical',
  'club',
  'comedy',
  'country',
  'dance',
  'dancehall',
  'death-metal',
  'deep-house',
  'detroit-techno',
  'disco',
  'disney',
  'drum-and-bass',
  'dub',
  'dubstep',
  'edm',
  'electro',
  'electronic',
  'emo',
  'folk',
  'forro',
  'french',
  'funk',
  'garage',
  'german',
  'gospel',
  'goth',
  'grindcore',
  'groove',
  'grunge',
  'guitar',
  'happy',
  'hard-rock',
  'hardcore',
  'hardstyle',
  'heavy-metal',
  'hip-hop',
  'holidays',
  'honky-tonk',
  'house',
  'idm',
  'indian',
  'indie',
  'indie-pop',
  'industrial',
  'iranian',
  'j-dance',
  'j-idol',
  'j-pop',
  'j-rock',
  'jazz',
  'k-pop',
  'kids',
  'latin',
  'latino',
  'malay',
  'mandopop',
  'metal',
  'metal-misc',
  'metalcore',


In [208]:
# Art-to-Music Mapping Concept

# Group 1: Abstract and Experimental
# Departments: Modern Art, Contemporary Art
# Genres: ambient, experimental, trip-hop, alt-rock, idm,
# Vibe: Abstract, innovative, and boundary-pushing, these genres provide a textured and atmospheric soundscape, perfect for the 
# unconventional nature of modern and contemporary works. The music offers an abstract exploration that reflects the introspective and 
# avant-garde qualities of this art.

# Group 2: Textiles and Cultural Arts
# Departments: Textiles, Arts of the Americas, Arts of Africa
# Genres: new-age, ethereal, drone, ambient, folk, americana, afro-beats
# Vibe: Rooted in cultural heritage and natural themes, these genres evoke an expansive, meditative quality. They emphasize tradition and 
# a connection to nature, while still feeling fresh, making them a fitting choice for textile art and culturally rich works from the
# Americas and Africa.

# Group 3: Historical and Classical Arts
# Departments: Painting and Sculpture of Europe, Applied Arts of Europe, Arts of the Ancient Mediterranean and Byzantium
# Genres: classical, ethereal, jazz, blues, classical
# Vibe: Timeless and sophisticated, these genres embody the rich history and craftsmanship of European and ancient Mediterranean art. 
# The music is regal and intricate, reflecting the elegance and historical depth of these classical and applied art forms.

# Group 4: Photography and Media
# Departments: Photography and Media, Prints and Drawings
# Genres: cinematic, dark-ambient, lo-fi, soundscapes, drone, ethereal, ambient, downtempo, soundscapes
# Vibe: Emotive and introspective, these genres are ideal for storytelling and capturing mood. The music enhances the narrative and 
# contemplative qualities of photography and intricate printmaking, creating an immersive experience that highlights the evocative 
# and detailed nature of these media.

# Group 5: Asian Art and Fusion
# Departments: Arts of Asia
# Genres: cantopop, k-pop, j-pop
# Vibe: This group showcases the vibrant and energetic sounds of contemporary pop music genres that resonate with the dynamic nature of 
# Asian art. Cantopop, K-pop, and J-pop reflect modern cultural influences and trends, creating an engaging soundtrack that enhances the 
# viewing experience.

In [209]:
# Mapping of art styles to music genres

art_to_music_map = {
    # Group 1: Abstract and Experimental
    "Modern Art": ["ambient", "experimental", "trip-hop", "alt-rock", "idm"],
    "Contemporary Art": ["ambient", "experimental", "trip-hop", "alt-rock", "idm"],
    
    # Group 2: Textiles and Cultural Arts
    "Textiles": ["new-age", "ethereal", "drone", "ambient"],
    "Arts of the Americas": ["folk", "americana"],
    "Arts of Africa": ["afrobeat", "african", "ethereal"],
    
    # Group 3: Historical and Classical Arts
    "Painting and Sculpture of Europe": ["classical", "ethereal"],
    "Applied Arts of Europe": ["jazz", "blues", "classical"],
    "Arts of the Ancient Mediterranean and Byzantium": ["classical","medieval"],
    
    # Group 4: Photography and Media
    "Photography and Media": ["cinematic", "dark-ambient", "lo-fi", "soundscapes", "drone"],
    "Prints and Drawings": ["ethereal", "ambient", "downtempo", "soundscapes"],
    
    # Group 5: Asian Art and Fusion
    "Arts of Asia": ["cantopop", "k-pop", "j-pop"]
}

In [210]:
# For loop through each art category and generate music recommendations based on genre mapping
for art_category, genre_seeds in art_to_music_map.items():
    print(f"\nMusic Recommendations for {art_category}:")

    # Spotify's API can accept up to 5 seed genres, so we take the first 5 from genre_seeds
    recommendations = sp.recommendations(seed_genres=genre_seeds[:5], limit=5)

    # Loop through the recommended tracks and print the details
    for track in recommendations['tracks']:
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        print(f" ♫ {track_name} by {artist_name}") 


Music Recommendations for Modern Art:
 ♫ New Beginning (Tidal Darkness) by Deaf Center
 ♫ System by Carbon Based Lifeforms
 ♫ Babel by Massive Attack
 ♫ On Top Of The World by Imagine Dragons
 ♫ Phase 09 - Sombrero by Solar Fields

Music Recommendations for Contemporary Art:
 ♫ Eggshell by Autechre
 ♫ Bad Girls by M.I.A.
 ♫ Hell Is Round The Corner by Tricky
 ♫ The Silent Orchestra by Biosphere
 ♫ Feels Like We Only Go Backwards by Tame Impala

Music Recommendations for Textiles:
 ♫ Spirit Within by Primal Instinct
 ♫ In The Court Of The Mermaid by Friedemann
 ♫ The Stolen Child by Loreena McKennitt
 ♫ Matta - Remastered 2005 by Brian Eno
 ♫ Principles Of Lust: Sadeness / Find Love / Sadeness (Reprise) by Enigma

Music Recommendations for Arts of the Americas:
 ♫ Truth by Alex Ebert
 ♫ Prïtoutïtze Planinata (Song from the Thracian Plain) by Bulgarian State Television Female Choir
 ♫ How Can You Swallow So Much Sleep by Bombay Bicycle Club
 ♫ Lunita by Calima
 ♫ Chico The American by F

In [211]:
# THE MAIN CHUNK! - MAKING IT WORK!

In [212]:
def display_virtual_exhibition_with_music(artworks_with_images_df, art_to_music_map, num_samples=5):
    # Randomly sample artworks to display
    sampled_artworks = artworks_with_images_df.sample(n=num_samples)
    all_recommended_tracks = []  # List to collect all recommended track URIs

    for _, artwork in sampled_artworks.iterrows():
        # Display the artwork details
        print(f"\nTitle: {artwork['title']}")
        print(f"Artist: {artwork['artist_display']}")
        print(f"Department: {artwork['department_title']}")
        display(Image(url=f"https://www.artic.edu/iiif/2/{artwork['image_id']}/full/843,/0/default.jpg", width=300))
        
        # Get the genres for the department
        department = artwork["department_title"]
        genre_seeds = art_to_music_map.get(department, [])
        
        # Generate music recommendations if genres are available
        if genre_seeds:
            print("Music Recommendations:")
            recommendations = sp.recommendations(seed_genres=genre_seeds[:5], limit=3)  # Limit to 3 tracks
            for track in recommendations['tracks']:
                track_name = track['name']
                artist_name = track['artists'][0]['name']
                track_uri = track['uri']
                all_recommended_tracks.append(track_uri)  # Collect track URI for playlist
                print(f" ♫ {track_name} by {artist_name}")
        else:
            print("No genre mapping found for this department.")
        
        # Separator for each artwork
        print("\n" + "-"*50 + "\n")
    
    # Return all collected track URIs
    return all_recommended_tracks

In [213]:
# Function to create a playlist and add collected tracks
def create_exhibition_playlist(track_uris):
    if not track_uris:
        print("No tracks to add to the playlist.")
        return
    
    # Create a new Spotify playlist for the exhibition
    playlist_name = "Virtual Art Exhibition Playlist"
    description = "A curated playlist featuring music to complement each artwork in the exhibition."
    playlist = sp.user_playlist_create(user=username, name=playlist_name, public=True, description=description)
    playlist_id = playlist['id']
    
    # Add tracks to the playlist
    sp.playlist_add_items(playlist_id, track_uris)
    print(f"Playlist '{playlist_name}' created with {len(track_uris)} tracks.")
    
    # Display the playlist URL
    playlist_url = playlist['external_urls']['spotify']
    print(f"Playlist URL: {playlist_url}")

In [214]:
# Run the function to display artworks, generate music recommendations, and create the playlist
track_uris = display_virtual_exhibition_with_music(artworks_with_images_df, art_to_music_map)
create_exhibition_playlist(track_uris)


Title: King Vulture
Artist: Meissen Porcelain Manufactory
German, founded 1710
Modeled by Johann Joachim Kändler
German, 1706-1775
Meissen, Germany
Department: Applied Arts of Europe


Music Recommendations:
 ♫ Fantasia on Greensleeves by Ralph Vaughan Williams
 ♫ Toccata And Fugue In D Minor, BWV 565 by Leopold Stokowski
 ♫ Working Man by Otis Rush

--------------------------------------------------


Title: Untitled (Butterfly Habitat)
Artist: Joseph Cornell
American, 1903–1972
Department: Modern Art


Music Recommendations:
 ♫ 14:31 by Global Communication
 ♫ Shake It Out by Florence + The Machine
 ♫ Sound The Alarm by Thievery Corporation

--------------------------------------------------


Title: Punch Pot
Artist: England, Staffordshire
Department: Applied Arts of Europe


Music Recommendations:
 ♫ As This Moment Slips Away by Joshua Redman
 ♫ Canticles of the Sky: Sky with Four Suns by John Luther Adams
 ♫ Hace Mucho, Mucho Tiempo by Javier Navarrete

--------------------------------------------------


Title: Coffer
Artist: Attributed to André-Charles Boulle (French, 1642–1732)
France, Paris
Department: Applied Arts of Europe


Music Recommendations:
 ♫ Chill Out (Things Gonna Change) feat. Carlos Santana by John Lee Hooker
 ♫ Call It Stormy Monday by Albert King
 ♫ She Moves Me by Muddy Waters

--------------------------------------------------


Title: Sideboard and Wine Cabinet
Artist: Designed by William Burges 
English, 1827–1881
Painted by Nathaniel Hubert John Westlake 
English, 1833–1921
Made by Harland & Fisher
London, England
Department: Applied Arts of Europe


Music Recommendations:
 ♫ Midnight Special by Lead Belly
 ♫ Touch Her Soft Lips and Part (From "Henry V") by William Walton
 ♫ Symphony No. 10 'Alla ricerca di Borromini', Op. 327: I. Adagio by Peter Maxwell Davies

--------------------------------------------------

Playlist 'Virtual Art Exhibition Playlist' created with 15 tracks.
Playlist URL: https://open.spotify.com/playlist/4ihPdxmiAE8HLrtEfgog5J
